In [43]:
from bs4 import BeautifulSoup
import requests
from config import scrape_key
import pandas as pd
from datetime import date, timedelta 

In [16]:
query_url = f"https://api.scrapingdog.com/scrape?api_key={scrape_key}&url=https://finance.yahoo.com/quote/AMZN?p=AMZN&.tsrc=fin-srch"
r = requests.get(query_url).text

In [17]:
#Beautiful Soup
soup = BeautifulSoup(r,'html.parser')
alldata = soup.find_all("tbody")

In [18]:
try:
 table1 = alldata[0].find_all('tr')
except:
 table1 = None
try:
 table2 = alldata[1].find_all('tr')
except:
 table2 = None

In [19]:
l={}
u=list()

In [20]:
for i in range(0,len(table1)):
 try:
   table1_td = table1[i].find_all('td')
 except:
   table1_td = None
 l[table1_td[0].text] = table1_td[1].text
 u.append(l)
 l={}

In [21]:
for i in range(0,len(table2)):
 try:
   table2_td = table2[i].find_all('td')
 except:
   table2_td = None
 l[table2_td[0].text] = table2_td[1].text
 u.append(l)
 l={}

In [29]:
dfobj = pd.DataFrame(u)
dfobj.iloc[0]['Previous Close']

'3,203.53'

In [35]:
a = date.today().strftime('%m-%d-%Y')
print(a)

12-03-2020


In [44]:
b = date.today() - timedelta(days = 1)